In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

from dataloader.boston_housing import BostonHousingData
from dataloader.concrete import ConcreteData
from dataloader.energy_efficiency import EnergyEfficiencyData
from experiment_setup import get_model, build_estimator
from model.mlp import MLP
from analysis.metrics import uq_accuracy, uq_ndcg, uq_nll
from uncertainty_estimator.masks import BasicMask, LHSMask, MirrorMask, DecorrelationMask, DPPMask

plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
config = {
    'random_seed': 43,
    'nn_runs': 100,
    'runs': 2,
    'model_runs': 1,
    'k_folds': 10,
    'verbose': False,
    'use_cache': False,
    'layers': [13, 64, 32, 32, 1],
    'epochs': 10_000,
    'acc_percentile': 0.1,
    'patience': 3,
    'dropout_train': 0.2,
    'dropout_uq': 0.4,
    'batch_size': 32,
    'dataset': 'energy_efficiency',
    'scale': True,
    'l2_reg': 1e-5
}

In [ ]:
# Load data
datasets = {
    'boston_housing': BostonHousingData,
    'concrete': ConcreteData,
    'energy_efficiency': EnergyEfficiencyData}

dataset = datasets[config['dataset']]()
    
model_paths = ["model/data/{}_{}.ckpt".format(config['dataset'], i) for i in range(config['model_runs'])]  
x_set, y_set = dataset.dataset('train')
x_test, y_test = dataset.dataset('val')
config['layers'][0] = x_set.shape[-1]

## k-fold

In [ ]:
# Normalize dataset
def scale(train, val):
    scaler = StandardScaler()
    scaler.fit(train)
    train = scaler.transform(train)
    val = scaler.transform(val)
    return train, val, scaler

# Train models
def plot_evaluations(model, x, y, x_, y_):
    predictions = model(x).cpu().numpy()
    plt.figure(figsize=(10, 10))
    plt.plot((min(y), max(y)), (min(y), max(y)))
    plt.scatter(model(x).cpu().numpy(), y)
    plt.scatter(model(x_).cpu().numpy(), y_)
    
# Evaluate different masks

def get_metrics(estimations, errors, acc_percentile=0.1):
    acc = uq_accuracy(estimations, errors, acc_percentile)
    ndcg = uq_ndcg(errors, estimations)
    nll = uq_nll(errors, estimations)
    return acc, ndcg, nll

def evaluate_masks(model, masks, x_val, y_val, y_scaler=None):
    predictions = model(x_val).cpu().numpy()
    errors = np.abs(predictions - y_val)
    results = []

    for name, mask in masks.items():
        estimator = build_estimator(
            'mcdue_masked', model, nn_runs=config['nn_runs'], dropout_mask=mask,
            dropout_rate=config['dropout_uq'])

        for run in range(config['runs']):
            estimations = estimator.estimate(x_val)
            acc, ndcg, nll = get_metrics(estimations, errors, config['acc_percentile'])
            results.append([acc, ndcg, nll, name])

            if hasattr(mask, 'reset'):
                mask.reset()
                
    return results

In [ ]:
# TODO
# make the split
# scale
# train
# evaluate
# regroup the results
# display the results

In [ ]:
masks = {
    'vanilla': None,
    'mirror_random': MirrorMask(),
    'decorrelating': DecorrelationMask(),
    'dpp': DPPMask()
}

In [ ]:
config['epochs'] = 3000
config['k_folds'] = 20
kfold = KFold(config['k_folds'], shuffle=True)

mask_results = [] 
for train_idx, val_idx in kfold.split(x_set):
    x_train, y_train = x_set[train_idx], y_set[train_idx]
    x_val, y_val = x_set[val_idx], y_set[val_idx]
    print('train', x_train.shape, y_train.shape, 'val', x_val.shape, y_val.shape)
    
    if config['scale']:
        x_train, x_val, _ = scale(x_train, x_val)
        y_train, y_val, y_scaler = scale(y_train, y_val)
    else:
        y_scaler = None
        
    
    model = MLP(config['layers'], l2_reg=config['l2_reg'])
    train_config = {k: config[k] for k in config if k in ['patience', 'dropout_rate', 'epochs', 'batch_size']}
    model.fit((x_train, y_train), (x_val, y_val), **train_config)
    plot_evaluations(model, x_train, y_train, x_val, y_val)
    results = evaluate_masks(model, masks, x_val, y_val)
    mask_results.extend(results)

In [ ]:
# Plot the results

mask_df = pd.DataFrame(mask_results, columns = ['acc', 'ndcg', 'nll', 'mask'])

plt.figure(figsize=(16, 6))
def boxplot(df, x_label, y_label, i, bottom=0, top=1):
    plt.subplot(1, 3, i)
    plt.xticks(rotation=45)
    if bottom is not None:
        plt.ylim(bottom, top)
    sns.boxplot(data=df, x=x_label, y=y_label)
    
boxplot(mask_df, 'mask', 'acc', 1, 0, 0.8)
boxplot(mask_df, 'mask', 'ndcg', 2, 0, 0.9)
boxplot(mask_df, 'mask', 'nll', 3, None)